# Slope

In [ ]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.windows import Window, from_bounds
from pyproj import Transformer
import matplotlib.pyplot as plt
from shapely.geometry import Point, box

ELEV_DATA_DIR = '../data/LF2020_Elev_220_CONUS'
FIRED_DATA_PATH = '../data/conus_ak/fired_conus_ak_2000_to_2024_events.csv' 

# --------------------------------------------------------------------------
# 1. DATA INTEGRATION: Combine FIRED with LANDFIRE topographic variables
# --------------------------------------------------------------------------

def load_fired_data(fired_path):
    """
    Load FIRED CONUS dataset and convert to GeoDataFrame with proper projection
    """
    # Load FIRED dataset
    print("Loading FIRED dataset...")
    fired_df = pd.read_csv(fired_path)

    # Convert ignition points to geometry
    geometry = [Point(x, y) for x, y in zip(fired_df['ig_utm_x'], fired_df['ig_utm_y'])]

    # Create GeoDataFrame with UTM projection (FIRED uses UTM)
    # Note: ig_utm_x and ig_utm_y are in UTM, but we need to know which UTM zone
    # For simplicity, assuming EPSG:5070 (Albers) which covers CONUS
    fired_gdf = gpd.GeoDataFrame(fired_df, geometry=geometry, crs="EPSG:5070")

    print(f"Loaded {len(fired_gdf)} wildfire events from FIRED dataset")
    return fired_gdf

def extract_topo_at_ignition(fired_gdf, raster_path, variable_name):
    """
    Extract topographic variable values at ignition points
    """
    print(f"Extracting {variable_name} at ignition points...")
    # Open raster dataset
    with rasterio.open(raster_path) as src:
        # Ensure GeoDataFrame is in same CRS as raster
        if fired_gdf.crs != src.crs:
            print(f"Reprojecting from {fired_gdf.crs} to {src.crs}")
            temp_gdf = fired_gdf.to_crs(src.crs)
        else:
            temp_gdf = fired_gdf.copy()

        # Create new column for this variable
        fired_gdf[variable_name] = np.nan

        # Extract values at points
        for idx, point in temp_gdf.iterrows():
            try:
                # Convert geometry to raster coordinates
                x, y = point.geometry.x, point.geometry.y
                row, col = src.index(x, y)

                # Get value from raster
                value = src.read(1)[row, col]

                # Assign to original dataframe
                if value != src.nodata:
                    fired_gdf.loc[idx, variable_name] = value
            except (IndexError, ValueError) as e:
                print(f"Warning: Could not extract {variable_name} for point {idx}: {e}")

    print(f"Extracted {variable_name} values with {fired_gdf[variable_name].notna().sum()} valid readings")
    return fired_gdf

def extract_topo_in_buffer(fired_gdf, raster_path, variable_name, buffer_distance=1000):
    """
    Extract topographic variable statistics within a buffer around ignition points
    """
    print(f"Extracting {variable_name} within {buffer_distance}m buffers...")

    # Create buffer in the GeoDataFrame's CRS
    fired_gdf_buffered = fired_gdf.copy()
    fired_gdf_buffered['geometry'] = fired_gdf.geometry.buffer(buffer_distance)

    # Open raster
    with rasterio.open(raster_path) as src:
        # Ensure same CRS
        if fired_gdf_buffered.crs != src.crs:
            print(f"Reprojecting buffers from {fired_gdf_buffered.crs} to {src.crs}")
            fired_gdf_buffered = fired_gdf_buffered.to_crs(src.crs)

        # Create columns for statistics
        stat_columns = [f"{variable_name}_mean", f"{variable_name}_min",
                       f"{variable_name}_max", f"{variable_name}_std"]

        for col in stat_columns:
            fired_gdf[col] = np.nan

        # Extract statistics for each buffer
        for idx, row in fired_gdf_buffered.iterrows():
            try:
                # Get bounds of buffer
                minx, miny, maxx, maxy = row.geometry.bounds

                # Create window from bounds
                window = from_bounds(minx, miny, maxx, maxy, src.transform)

                # Read data within window
                data = src.read(1, window=window)

                # Mask nodata values
                masked_data = np.ma.masked_equal(data, src.nodata)

                # Calculate statistics
                if not masked_data.mask.all():  # Check if we have valid data
                    fired_gdf.loc[idx, f"{variable_name}_mean"] = float(masked_data.mean())
                    fired_gdf.loc[idx, f"{variable_name}_min"] = float(masked_data.min())
                    fired_gdf.loc[idx, f"{variable_name}_max"] = float(masked_data.max())
                    fired_gdf.loc[idx, f"{variable_name}_std"] = float(masked_data.std())
            except Exception as e:
                print(f"Warning: Error processing buffer {idx}: {e}")

    return fired_gdf

def integrate_fired_with_topo(fired_path, elev_raster, slope_raster=None, aspect_raster=None, buffer_distance=1000):
    """
    Main function to integrate FIRED data with LANDFIRE topographic data
    """
    # Load FIRED data
    fired_gdf = load_fired_data(fired_path)

    # Extract elevation at ignition points
    fired_gdf = extract_topo_at_ignition(fired_gdf, elev_raster, "elevation")

    # Extract elevation stats within buffer
    fired_gdf = extract_topo_in_buffer(fired_gdf, elev_raster, "elevation", buffer_distance)

    # If slope raster is provided, extract slope data
    if slope_raster:
        fired_gdf = extract_topo_at_ignition(fired_gdf, slope_raster, "slope")
        fired_gdf = extract_topo_in_buffer(fired_gdf, slope_raster, "slope", buffer_distance)
    else:
        # Calculate slope from elevation if slope raster not provided
        print("Slope raster not provided. Slope statistics will not be available.")

    # If aspect raster is provided, extract aspect data
    if aspect_raster:
        fired_gdf = extract_topo_at_ignition(fired_gdf, aspect_raster, "aspect")
        fired_gdf = extract_topo_in_buffer(fired_gdf, aspect_raster, "aspect", buffer_distance)
    else:
        print("Aspect raster not provided. Aspect statistics will not be available.")

    return fired_gdf

# --------------------------------------------------------------------------
# 2. FEATURE ENGINEERING: Create derived topographic features
# --------------------------------------------------------------------------

def engineer_topo_features(df):
    """
    Create derived features from topographic variables
    """
    print("Engineering topographic features...")

    # Copy the dataframe to avoid modifying the original
    df_featured = df.copy()

    # --- Slope-related features ---
    if 'slope' in df_featured.columns:
        # Categorize slope into classes
        bins = [0, 5, 15, 30, 100]
        labels = ['Flat', 'Moderate', 'Steep', 'Very Steep']
        df_featured['slope_class'] = pd.cut(df_featured['slope'], bins=bins, labels=labels)

        # Binary indicators for slope classes
        for label in labels:
            df_featured[f'slope_is_{label.lower().replace(" ", "_")}'] = (df_featured['slope_class'] == label).astype(int)

    # --- Aspect-related features ---
    if 'aspect' in df_featured.columns:
        # Convert aspect to cardinal/ordinal directions
        def aspect_to_direction(aspect):
            if pd.isna(aspect):
                return np.nan

            # Adjust aspect values to start from North = 0
            directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
            # Convert 0-360 degrees to 0-7 index
            idx = round(((aspect % 360) / 45)) % 8
            return directions[idx]

        df_featured['aspect_direction'] = df_featured['aspect'].apply(aspect_to_direction)

        # Create binary indicators for each direction
        for direction in ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']:
            df_featured[f'aspect_{direction}'] = (df_featured['aspect_direction'] == direction).astype(int)

        # Group aspects into quadrants
        df_featured['aspect_quadrant'] = df_featured['aspect_direction'].map({
            'N': 'North', 'NE': 'Northeast', 'E': 'East', 'SE': 'Southeast',
            'S': 'South', 'SW': 'Southwest', 'W': 'West', 'NW': 'Northwest'
        })

        # Simplified north-south indicator (north = 0, south = 1)
        north_dirs = ['N', 'NE', 'NW']
        south_dirs = ['S', 'SE', 'SW']
        df_featured['aspect_south_facing'] = df_featured['aspect_direction'].apply(
            lambda x: 1 if x in south_dirs else (0 if x in north_dirs else 0.5)
        )

    # --- Combined slope-aspect features ---
    if 'slope' in df_featured.columns and 'aspect' in df_featured.columns:
        # Potential solar radiation (simplified model)
        def potential_solar(row, latitude=40):
            """Simplified solar radiation model based on slope and aspect"""
            if pd.isna(row['slope']) or pd.isna(row['aspect']):
                return np.nan

            slope_rad = np.radians(row['slope'])
            aspect_rad = np.radians(row['aspect'])
            lat_rad = np.radians(latitude)

            # Higher values for south-facing slopes in Northern Hemisphere
            return np.cos(slope_rad) * np.cos(lat_rad) + np.sin(slope_rad) * np.sin(lat_rad) * np.cos(aspect_rad)

        df_featured['potential_solar'] = df_featured.apply(potential_solar, axis=1)

        # Interaction: steep south-facing slopes
        if 'aspect_S' in df_featured.columns:
            df_featured['steep_south_slope'] = df_featured['slope'] * df_featured['aspect_S']

    # --- Elevation variability features ---
    if all(col in df_featured.columns for col in ['elevation_min', 'elevation_max', 'elevation_std']):
        # Calculate relief (max - min elevation) within buffer
        df_featured['relief'] = df_featured['elevation_max'] - df_featured['elevation_min']

        # Normalize relief by buffer area (rough terrain roughness proxy)
        # Assuming buffer_area in km²
        buffer_area = np.pi * (1.0**2)  # Area of 1km buffer in km²
        df_featured['terrain_roughness'] = df_featured['relief'] / buffer_area

        # High vs low elevation binary indicator (using median as threshold)
        median_elev = df_featured['elevation'].median()
        df_featured['high_elevation'] = (df_featured['elevation'] > median_elev).astype(int)

    # --- Fire spread specific features ---
    # Calculate topographic alignment with dominant wind direction
    if 'aspect' in df_featured.columns:
        wind_direction = 270  # Example: wind from the west

        # Calculate alignment (0° = perfectly aligned, 180° = opposing)
        df_featured['wind_alignment'] = np.abs((df_featured['aspect'] - wind_direction + 180) % 360 - 180)

        # Normalize to 0-1 range (1 = perfectly aligned with wind)
        df_featured['wind_alignment_norm'] = 1 - (df_featured['wind_alignment'] / 180)

    # Add fire season indicator (if ig_date is available)
    if 'ig_date' in df_featured.columns:
        # Convert to datetime
        df_featured['ig_date'] = pd.to_datetime(df_featured['ig_date'])

        # Extract month
        df_featured['ig_month'] = df_featured['ig_date'].dt.month

        # Define fire seasons (adjust for your region)
        df_featured['fire_season'] = pd.cut(
            df_featured['ig_month'],
            bins=[0, 4, 10, 13],  # 1-4: spring, 5-10: summer/fall, 11-12: winter
            labels=['Spring', 'Peak', 'Winter']
        )

    # --- Custom feature: slope position ---
    if 'elevation' in df_featured.columns and 'elevation_mean' in df_featured.columns:
        # Relative elevation position
        df_featured['rel_position'] = (df_featured['elevation'] - df_featured['elevation_min']) / \
                                     (df_featured['elevation_max'] - df_featured['elevation_min'])

        # Categorize position
        def slope_position(rel_pos):
            if pd.isna(rel_pos):
                return np.nan
            elif rel_pos < 0.33:
                return 'Lower'
            elif rel_pos > 0.66:
                return 'Upper'
            else:
                return 'Mid'

        df_featured['slope_position'] = df_featured['rel_position'].apply(slope_position)

    print(f"Created {len(df_featured.columns) - len(df.columns)} new features")
    return df_featured

# --------------------------------------------------------------------------
# Main execution function
# --------------------------------------------------------------------------

def process_fired_topo_data():
    """
    Main function to process FIRED data with LANDFIRE topographic data
    """
    # Define file paths
    fired_path = FIRED_DATA_PATH

    # Use your existing elevation data path
    elevation_file = os.path.join(ELEV_DATA_DIR, "Tif/LC20_Elev_220.tif")

    # Look for slope and aspect files (assuming they follow standard naming)
    # If not available, they'll be set to None
    slope_file = os.path.join(ELEV_DATA_DIR, "Tif/LC20_Slope.tif")
    if not os.path.exists(slope_file):
        slope_file = None
        print("Slope raster not found at expected path")

    aspect_file = os.path.join(ELEV_DATA_DIR, "Tif/LC20_Aspect.tif")
    if not os.path.exists(aspect_file):
        aspect_file = None
        print("Aspect raster not found at expected path")

    # 1. Integrate FIRED with topographic data
    print("\n--- STEP 1: Data Integration ---")
    integrated_data = integrate_fired_with_topo(
        fired_path,
        elevation_file,
        slope_file,
        aspect_file,
        buffer_distance=1000  # 1km buffer
    )

    # 2. Engineer topographic features
    print("\n--- STEP 2: Feature Engineering ---")
    final_data = engineer_topo_features(integrated_data)

    # Save the processed data
    output_path = "fired_topo_features.csv"
    final_data.to_csv(output_path, index=False)
    print(f"\nProcessed data saved to {output_path}")

    # Also save a GeoJSON for spatial analysis
    geojson_path = "fired_topo_features.geojson"
    final_data.to_file(geojson_path, driver="GeoJSON")
    print(f"Spatial data saved to {geojson_path}")

    return final_data

# --------------------------------------------------------------------------
# Visualization helper functions
# --------------------------------------------------------------------------

def visualize_topo_fire_relationship(df, x_var, y_var='fsr_km2_dy', hue_var=None):
    """
    Create scatter plot to visualize relationship between topographic variable and fire spread rate
    """
    plt.figure(figsize=(10, 6))

    if hue_var and hue_var in df.columns:
        # Categorical color mapping
        if df[hue_var].dtype.name == 'category' or df[hue_var].nunique() < 10:
            sns.scatterplot(x=x_var, y=y_var, hue=hue_var, data=df)
        else:
            # Continuous color mapping
            scatter = plt.scatter(
                df[x_var], df[y_var],
                c=df[hue_var],
                cmap='viridis',
                alpha=0.7
            )
            plt.colorbar(scatter, label=hue_var)
    else:
        sns.scatterplot(x=x_var, y=y_var, data=df)

    plt.title(f"Relationship between {x_var} and Fire Spread Rate")
    plt.xlabel(x_var)
    plt.ylabel("Fire Spread Rate (km²/day)")
    plt.tight_layout()

    return plt

# Execute if run as script
if __name__ == "__main__":
    try:
        import seaborn as sns
        processed_data = process_fired_topo_data()

        # Example: Create a visualization of slope vs. fire spread rate
        if 'slope' in processed_data.columns and 'fsr_km2_dy' in processed_data.columns:
            visualize_topo_fire_relationship(
                processed_data,
                x_var='slope',
                hue_var='aspect_direction'
            )
            plt.savefig("slope_vs_spread_rate.png")
            print("Visualization saved to slope_vs_spread_rate.png")
    except Exception as e:
        print(f"Error during processing: {e}")

Slope raster not found at expected path
Aspect raster not found at expected path

--- STEP 1: Data Integration ---
Loading FIRED dataset...
Loaded 342731 wildfire events from FIRED dataset
Extracting elevation at ignition points...
